In [1]:
!pip install tensorflow
!pip install sklearn
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf


In [2]:
# while(True):
#     cap = cv2.VideoCapture(0)
#     # Capture frame-by-frame
#     ret, frame = cap.read()
# 
#     # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# 
#     # Display the resulting frame
#     cv2.imshow('frame',gray)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# 
# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

In [2]:
DATA_DIR1 = "Rock-Paper-Scissors/train"
CATEGORIES = ["paper", "rock", "scissors"]
IMG_WIDTH = 75
IMG_HEIGHT = 50
BATCH_SIZE = 150

def load_data(data_dir1, CATEGORIES, img_width, img_height):
    X = []
    y = []
    index = 0

    for category in CATEGORIES:
        one_hot = np.zeros(len(CATEGORIES))  # to encode the class as a one hot vector
        one_hot[index] = 1
        path = os.path.join(data_dir1, category)
        index += 1

        for img in os.listdir(path): # get all images in the path
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.resize(img_arr, (img_width, img_height))
            img_arr = np.asarray(img_arr)
            X.append(img_arr)
            y.append(one_hot)

    return X, y

X, y = load_data(DATA_DIR1, CATEGORIES, IMG_WIDTH, IMG_HEIGHT)

In [3]:
DATA_DIR2 = "rockpaperscissors"
CATEGORIES = ["paper", "rock", "scissors"]
IMG_WIDTH = 75
IMG_HEIGHT = 50
BATCH_SIZE = 100

def load_data(data_dir2, CATEGORIES, img_width, img_height):
    X = []
    y = []
    index = 0
    
    for category in CATEGORIES:
        one_hot = np.zeros(len(CATEGORIES))  # to encode the class as a one hot vector
        one_hot[index] = 1
        path = os.path.join(data_dir2, category)
        index += 1
        
        for img in os.listdir(path): # get all images in the path
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.resize(img_arr, (img_width, img_height))
            img_arr = np.asarray(img_arr)
            X.append(img_arr)
            y.append(one_hot)
            
    return X, y

X2, y2 = load_data(DATA_DIR2, CATEGORIES, IMG_WIDTH, IMG_HEIGHT)
X += X2
y+= y2

In [4]:
# Divide our data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

# Normalize our data
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because we are using greyscale, we only have a single channel - RGB colour images would have 3
X_train = X_train.reshape(X_train.shape[0], IMG_WIDTH, IMG_HEIGHT, 1)
X_test = X_test.reshape(X_test.shape[0], IMG_WIDTH, IMG_HEIGHT, 1)
input_shape = (IMG_WIDTH, IMG_HEIGHT, 1)

# convert the data to the right type
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# X_train /= 50
# X_test /= 50
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

pass

x_train shape: (3766, 75, 50, 1)
3766 train samples
942 test samples


In [5]:
# Build Model


# Set the model type as sequential 
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(34, kernel_size=(3, 3), strides=(1,1), activation="relu", input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(1,1)))

model.add(tf.keras.layers.Conv2D(34, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(45, activation="relu"))
model.add(tf.keras.layers.Dropout(.45))
model.add(tf.keras.layers.Dense(len(CATEGORIES), activation="softmax"))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(lr=0.02),
              metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train),
          batch_size=BATCH_SIZE,
          epochs=100,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)))

Train on 3766 samples, validate on 942 samples
Epoch 1/100
3766/3766 [==============================] - 33s 9ms/sample - loss: 1.0989 - accuracy: 0.3380 - val_loss: 1.0979 - val_accuracy: 0.3652
Epoch 2/100
3766/3766 [==============================] - 32s 9ms/sample - loss: 1.0966 - accuracy: 0.3603 - val_loss: 1.0948 - val_accuracy: 0.3758
Epoch 3/100
3766/3766 [==============================] - 33s 9ms/sample - loss: 1.0950 - accuracy: 0.3712 - val_loss: 1.0926 - val_accuracy: 0.4236
Epoch 4/100
3766/3766 [==============================] - 33s 9ms/sample - loss: 1.0932 - accuracy: 0.3885 - val_loss: 1.0905 - val_accuracy: 0.4926
Epoch 5/100
3766/3766 [==============================] - 33s 9ms/sample - loss: 1.0904 - accuracy: 0.4079 - val_loss: 1.0889 - val_accuracy: 0.4469
Epoch 6/100
3766/3766 [==============================] - 33s 9ms/sample - loss: 1.0880 - accuracy: 0.4047 - val_loss: 1.0870 - val_accuracy: 0.3652
Epoch 7/100
3766/3766 [==============================] - 32s 9ms/

Epoch 56/100
3766/3766 [==============================] - 32s 8ms/sample - loss: 0.2427 - accuracy: 0.9153 - val_loss: 0.1968 - val_accuracy: 0.9352
Epoch 57/100
3766/3766 [==============================] - 33s 9ms/sample - loss: 0.2365 - accuracy: 0.9187 - val_loss: 0.1819 - val_accuracy: 0.9406
Epoch 58/100
3766/3766 [==============================] - 40s 11ms/sample - loss: 0.2244 - accuracy: 0.9251 - val_loss: 0.1808 - val_accuracy: 0.9416
Epoch 59/100
3766/3766 [==============================] - 45s 12ms/sample - loss: 0.2227 - accuracy: 0.9267 - val_loss: 0.1996 - val_accuracy: 0.9352
Epoch 60/100
3766/3766 [==============================] - 40s 11ms/sample - loss: 0.2085 - accuracy: 0.9302 - val_loss: 0.1711 - val_accuracy: 0.9448
Epoch 61/100
3766/3766 [==============================] - 40s 11ms/sample - loss: 0.2144 - accuracy: 0.9315 - val_loss: 0.1716 - val_accuracy: 0.9448
Epoch 62/100
3766/3766 [==============================] - 47s 12ms/sample - loss: 0.1963 - accuracy: 0

In [ ]:
# function to parse single image
def load_image(image_path):
   img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
   img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
   img = np.asarray(img)
   #normalize image
   img = tf.keras.utils.normalize(img, axis=1)
   #reshape into tensor
   img = img.reshape(1, IMG_WIDTH, IMG_HEIGHT, 1)
   return img

In [ ]:
# test model with selected image
rock = "rockpaperscissors/rock/tGBAO9oyBYRD7XXS.png"
paper = "rockpaperscissors/paper/tiS1ECooPwR5IvFK.png"
scissors = "rockpaperscissors/scissors/pZ9zE5ahoKZhIZqm.png"
catagories = ['paper', 'rock', 'scissors']
test_img = [rock, paper, scissors]
selector = 0
prediction = classifier.predict_classes(load_image(test_img[selector]))
plt.imshow(cv2.imread(test_img[x]))
print("predicted action is", catagories[prediction[0]])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])